In [1]:
import findspark
findspark.init()
import json
import sys
import seaborn as sns 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime

import pyspark.sql.functions as fn
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression


#from pyspark.ml.recommendation import ALS, ALSModel
#from pyspark.mllib.recommendation import MatrixFactorizationModel
from pyspark.sql import Row, SQLContext,SparkSession
import os

parking = "TresCrucesShopping"
hadoop_home = "/opt/hadoop/hadoop-3.3.2"

os.environ['HADOOP_USER_NAME'] = "hadoop"
os.environ['HADOOP_CONF_DIR'] = hadoop_home + "/etc/hadoop"
os.environ['YARN_CONF_DIR'] = hadoop_home + "/etc/hadoop"

warnings.filterwarnings("ignore")

master = "local[*]"
#master = "yarn"

# Spark session & context
spark = (SparkSession
         .builder
         .master(master)
         .appName('RL-GenerateJsonSummary')
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")
sc = spark.sparkContext

22/08/05 10:31:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/05 10:31:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/05 10:31:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/05 10:31:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
spark

In [3]:
now = datetime.now()

In [8]:
df_parking_machine_learning = spark \
    .read \
    .parquet(f"hdfs://hadoop-namenode:9000/predictions/Parkings/{parking}/year={now.year}/month={now.month - 1}/day={now.day}/hour={now.hour}/minute={now.minute}")

In [9]:
df_parking_machine_learning = df_parking_machine_learning \
                                .withColumnRenamed("area_ocupation_percentage_target", "area_occupation_percentage_target")

In [10]:
df_parking_machine_learning.show()


+--------+-------+---------------------------------+--------------------------+
|level_id|area_id|area_occupation_percentage_target|area_occupation_percentage|
+--------+-------+---------------------------------+--------------------------+
|       1|      1|               20.494242060714598|                        20|
|       1|      2|               30.327811455448266|                        30|
|       1|      3|                59.82851963964928|                        60|
|       2|      1|                40.16138085018194|                        40|
|       2|      2|                49.99495024491561|                        50|
|       2|      3|               10.660672665980933|                        10|
|       2|      4|               10.660672665980933|                        10|
|       3|      1|                59.82851963964928|                        60|
|       3|      2|                59.82851963964928|                        60|
+--------+-------+----------------------

In [34]:
with open("/opt/smart-parking/Python/MachineLearning/ParkingSummary.json") as file:
    json_file = json.load(file)

In [36]:
data_dict = {}
for row in df_parking_machine_learning.rdd.collect():
    if row['level_id'] in data_dict:
        data_dict[row['level_id']].append({'area_id': row['area_id'], 
                                'area_occupation_percentage_target': round(row['area_occupation_percentage_target'])})
    else:
        data_dict.update({row['level_id']:[]})
        data_dict[row['level_id']].append({'area_id': row['area_id'], 
                                'area_occupation_percentage_target': round(row['area_occupation_percentage_target'])})
        

In [38]:
levels_object = []

for k, v in data_dict.items():
    levels_object.append({'level_id': k, 'areas': v})


In [41]:
json_file['levels'] = levels_object

In [47]:
print(json.dumps(json_file))

{"coefficients": {"0to20Coef": "1.1", "20to40Coef": "1.2", "40to60Coef": "1.4", "60to80Coef": "1.8", "80to100Coef": "2"}, "hasDynamicPrice": true, "levels": [{"level_id": 1, "areas": [{"area_id": 1, "area_occupation_percentage_target": 20}, {"area_id": 2, "area_occupation_percentage_target": 30}, {"area_id": 3, "area_occupation_percentage_target": 60}]}, {"level_id": 2, "areas": [{"area_id": 1, "area_occupation_percentage_target": 40}, {"area_id": 2, "area_occupation_percentage_target": 50}, {"area_id": 3, "area_occupation_percentage_target": 11}, {"area_id": 4, "area_occupation_percentage_target": 11}]}, {"level_id": 3, "areas": [{"area_id": 1, "area_occupation_percentage_target": 60}, {"area_id": 2, "area_occupation_percentage_target": 60}]}]}


In [48]:
with open(f"/opt/data/GetData/Parkings/{parking}/MachineLearning/ParkingSummary.json", 'w') as f:
    f.write(json.dumps(json_file)) 

FileNotFoundError: [Errno 2] No such file or directory: '/opt/data/GetData/Parkings/TresCrucesShopping/MachineLearning/ParkingSummary.json'